In [1]:
import sys

sys.path.append("../")

from funciones import *

In [2]:

img_width=224
img_height=224

#CARGANDO LOS DATASETS
path_root='../../Dataset Real Bananos/'

path_test=path_root+'test'
imagenes_test=[]
directorio_test=[]
dircount_test=cargar_dataset(imagenes_test,directorio_test,path_test)

path_train=path_root+'train'
imagenes_train=[]
directorio_train=[]
dircount_train=cargar_dataset(imagenes_train,directorio_train,path_train)

path_validation=path_root+'validation'
imagenes_validation=[]
directorio_validation=[]

dircount_validation=cargar_dataset(imagenes_validation,directorio_validation,path_validation)

leyendo imagenes de  c:\Users\leone\Documents\Mi_Fase2_Bananos\pruebas con Inception ResnetV2\../../Dataset Real Bananos/test\
c:\Users\leone\Documents\Mi_Fase2_Bananos\pruebas con Inception ResnetV2\../../Dataset Real Bananos/test\Class A 1
c:\Users\leone\Documents\Mi_Fase2_Bananos\pruebas con Inception ResnetV2\../../Dataset Real Bananos/test\Class B 208
c:\Users\leone\Documents\Mi_Fase2_Bananos\pruebas con Inception ResnetV2\../../Dataset Real Bananos/test\Class C 187
c:\Users\leone\Documents\Mi_Fase2_Bananos\pruebas con Inception ResnetV2\../../Dataset Real Bananos/test\Class D 191
Directorios leidos: 4
Imagenes en cada directorio [209, 187, 191, 213]
suma Total de imagenes en subdirs: 800
leyendo imagenes de  c:\Users\leone\Documents\Mi_Fase2_Bananos\pruebas con Inception ResnetV2\../../Dataset Real Bananos/train\
c:\Users\leone\Documents\Mi_Fase2_Bananos\pruebas con Inception ResnetV2\../../Dataset Real Bananos/train\Class A 1
c:\Users\leone\Documents\Mi_Fase2_Bananos\pruebas con

In [3]:
'''
#Imprimiendo las primeras 9 imagenes
for i in range(9):
	# define subplot
	pyplot.subplot(330 + 1 + i)
	# plot raw pixel data
	pyplot.imshow(images[i])
# show the figure
pyplot.show()
'''


'\n#Imprimiendo las primeras 9 imagenes\nfor i in range(9):\n\t# define subplot\n\tpyplot.subplot(330 + 1 + i)\n\t# plot raw pixel data\n\tpyplot.imshow(images[i])\n# show the figure\npyplot.show()\n'

In [4]:
X_train,y_train=agregar_etiquetas(directorio_train,dircount_train,imagenes_train)

X_test,y_test=agregar_etiquetas(directorio_test,dircount_test,imagenes_test)

X_validation,y_validation=agregar_etiquetas(directorio_validation,dircount_validation,imagenes_validation)

Cantidad etiquetas creadas:  2384
0 Class A
1 Class B
2 Class C
3 Class D
Cantidad etiquetas creadas:  800
0 Class A
1 Class B
2 Class C
3 Class D
Cantidad etiquetas creadas:  795
0 Class A
1 Class B
2 Class C
3 Class D


In [5]:
# Find the unique numbers from the train labels
classes = np.unique(y_test)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Total number of outputs :  4
Output classes :  [0 1 2 3]


In [6]:
y_one_hot_test=normalizar_and_one_hot_encoding(X_test,y_test)

y_one_hot_train=normalizar_and_one_hot_encoding(X_train,y_train)

y_one_hot_validation=normalizar_and_one_hot_encoding(X_validation,y_validation)
print('Original label Test:', y_test[0])
print('After conversion to one-hot Test:', y_one_hot_test[0])
print("\n")
print('Original label Train:', y_train[0])
print('After conversion to one-hot Train:', y_one_hot_train[0])
print("\n")
print('Original label Validation:', y_validation[0])
print('After conversion to one-hot Validation:', y_one_hot_validation[0])
print("\n")
print("*******************Test****************")
print(X_test.shape, y_one_hot_test.shape)
print("\n")
print("*******************Train****************")
print(X_train.shape, y_one_hot_train.shape)
print("\n")
print("*******************Validation****************")
print(X_validation.shape, y_one_hot_validation.shape)


Original label Test: 0
After conversion to one-hot Test: [1. 0. 0. 0.]


Original label Train: 0
After conversion to one-hot Train: [1. 0. 0. 0.]


Original label Validation: 0
After conversion to one-hot Validation: [1. 0. 0. 0.]


*******************Test****************
(800, 224, 224, 3) (800, 4)


*******************Train****************
(2384, 224, 224, 3) (2384, 4)


*******************Validation****************
(795, 224, 224, 3) (795, 4)


In [7]:
#Construccion del modelo
INIT_LR = 1e-3
batch_size=50
epochs=100
inception_arch=InceptionResNetV2(input_shape=(img_width,img_height,3),
                                 weights=None, #aqui se puede aplicarTransfer Learning
                                 include_top=False)
model = Sequential()
model.add(inception_arch)
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(nClasses, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 5, 5, 1536)        54336736  
_________________________________________________________________
flatten (Flatten)            (None, 38400)             0         
_________________________________________________________________
dropout (Dropout)            (None, 38400)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              39322624  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0

In [8]:
# Use some functions from tensorflow_docs
#!pip install git+https://github.com/tensorflow/docs > /dev/null

In [9]:
#EMPEZANDO EL ENTRENAMIENTO

#se termina cuando el accuracy deja de mejorar
#import tensorflow_docs as tfdocs
#import tensorflow_docs.plots
#import tensorflow_docs.modeling

#early_stop = keras.callbacks.EarlyStopping(monitor='accuracy', patience=10)

'''
import tensorflow_docs as tfdocs
early_stop = keras.callbacks.EarlyStopping(monitor='val_mae', patience=10)

early_history = model.fit(normed_train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=2,
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])
                    # callbacks=[tfdocs.modeling.EpochDots()])
                    '''
history=model.fit(X_train, y_one_hot_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  #callbacks=[early_stop, tfdocs.modeling.EpochDots()],
                  verbose=1,
                  validation_data=(X_validation, y_one_hot_validation))


Epoch 1/100
 1/48 [..............................] - ETA: 0s - loss: 2.0689 - accuracy: 0.2400

KeyboardInterrupt: 

In [ ]:
_, acc = model.evaluate(X_test, y_one_hot_test, verbose=0)
	print('> %.3f' % (acc * 100.0))

In [ ]:
path_modelo='./modelos/ipception_resnetV2.h5'
model.save(path_modelo)

In [ ]:

# entry point, run the example
run_example('../imagenes de prueba/banana1_C.jpg',path_modelo)